In [ ]:
import cv2
import numpy as np
import os

def is_hand_present(image_path, threshold=0.3):
    """
    이미지의 왼쪽 영역에서 손의 존재 여부를 판단합니다.

    :param image_path: 이미지 경로
    :param threshold: 피부 색 영역 비율 임계값 (0~1)
    :return: 손이 존재하면 True, 아니면 False
    """
    # 이미지 읽기
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Cannot read image: {image_path}")
    
    # 이미지의 왼쪽 30% 영역 추출
    height, width, _ = image.shape
    left_region = image[:, :int(width * 0.3)]

    # HSV로 변환
    hsv = cv2.cvtColor(left_region, cv2.COLOR_BGR2HSV)

    # 피부 색상 범위 지정 (HSV)
    lower_skin = np.array([0, 30, 60], dtype=np.uint8)
    upper_skin = np.array([20, 150, 255], dtype=np.uint8)

    # 피부 색상 영역 마스크 생성
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # 마스크 비율 계산
    skin_area_ratio = np.sum(mask > 0) / mask.size

    return skin_area_ratio > threshold

def check_and_delete(input_image_folder, input_label_folder, threshold=0.3):
    """
    손이 감지된 이미지와 동일한 이름의 JSON 파일을 삭제합니다.

    :param input_image_folder: 입력 이미지 폴더 경로
    :param input_label_folder: JSON 파일 폴더 경로
    :param threshold: 손 감지 임계값
    """
    total_images = 0
    hand_detected_images = 0

    for file_name in os.listdir(input_image_folder):
        image_path = os.path.join(input_image_folder, file_name)
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            total_images += 1
            try:
                if is_hand_present(image_path, threshold):
                    hand_detected_images += 1

                    # # 이미지 삭제
                    # os.remove(image_path)
                    # print(f"Deleted image: {file_name}")

                    # # JSON 파일 삭제
                    # json_file = os.path.splitext(file_name)[0] + '.json'
                    # json_path = os.path.join(input_label_folder, json_file)
                    # if os.path.exists(json_path):
                    #     os.remove(json_path)
                    #     print(f"Deleted label: {json_file}")
                    # else:
                    #     print(f"Label file not found: {json_file}")
            except Exception as e:
                print(f"Error processing {file_name}: {e}")

    # 결과 출력
    print("\n--- Summary ---")
    print(f"Total images processed: {total_images}")
    print(f"Images with hands detected and deleted: {hand_detected_images}")
    print(f"Images without hands: {total_images - hand_detected_images}")

if __name__ == "__main__":
    # 입력 폴더 설정
    input_image_folder = "/home/knuvi/Desktop/song/cucumber-image/data/whole_oi/images"
    input_label_folder = "/home/knuvi/Desktop/song/cucumber-image/data/whole_oi/labels"

    # 실행
    check_and_delete(input_image_folder, input_label_folder, threshold=0.3)
